In [1]:
import numpy as np 
import pandas as pd 
import matplotlib as plt 
import seaborn as sns 
import tensorflow as tf
from tensorflow import keras
import tensorflow.python.platform.build_info as build
from database_connection import DBConn

In [2]:
# check gpu availability 
print(f"TF is cuda build: {build.build_info['is_cuda_build']}")
print(f"Device list: {tf.config.list_physical_devices('GPU')}")

TF is cuda build: True
Device list: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# setup database connection 
HOST = 'localhost'
USER = 'db_user'
PORT = 5438
DATABASE = 'shot_db'
PASSWORD = 'LetMeIn'
TABLE_NAME = 'shot_data_table'

db = DBConn()

In [14]:
# grab a sample of the data to check out 
value = db.query(f"SELECT * FROM {TABLE_NAME} WHERE shot_event<>'BLOCK' ORDER BY id ASC LIMIT 25")
value.columns 

Index(['id', 'game_id', 'season', 'game_date', 'game_period', 'shot_event',
       'seconds_elapsed', 'strength', 'strength_2', 'event_team', 'away_team',
       'home_team', 'position', 'shoots', 'player1', 'player2', 'player3',
       'away_player1', 'away_player2', 'away_player3', 'away_player4',
       'away_player5', 'away_player6', 'home_player1', 'home_player2',
       'home_player3', 'home_player4', 'home_player5', 'home_player6',
       'away_players', 'home_players', 'away_score', 'home_score',
       'away_goalie', 'home_goalie', 'home_coach', 'away_coach', 'event_zone',
       'x', 'y', 'is_home', 'goalie', 'catches', 'shot_type', 'loc', 'corsi',
       'fenwick', 'shot', 'goal', 'empty_net', 'xg', 'event_distance',
       'event_angle'],
      dtype='object')

In [15]:
value

,id,game_id,season,game_date,game_period,shot_event,seconds_elapsed,strength,strength_2,event_team,...,shot_type,loc,corsi,fenwick,shot,goal,empty_net,xg,event_distance,event_angle
0,1,2007020001,20072008,2007-09-29,1,SHOT,17,5x5,EV,L.A,...,Wrist,1,1,1,1,0,0,0.069568,21.954498,30.068583
1,2,2007020001,20072008,2007-09-29,1,MISS,52,5x5,EV,ANA,...,Wrist,1,1,1,0,0,0,0.014901,51.923020,-15.642246
2,3,2007020001,20072008,2007-09-29,1,SHOT,86,5x5,EV,ANA,...,Backhand,1,1,1,1,0,0,0.051133,22.472205,20.854458
3,4,2007020001,20072008,2007-09-29,1,SHOT,188,4x4,EV,L.A,...,Wrist,1,1,1,1,0,0,0.042231,31.827661,43.726970
4,5,2007020001,20072008,2007-09-29,1,SHOT,242,4x5,PP,ANA,...,Wrist,1,1,1,1,0,0,0.031222,42.438190,-34.438989
5,6,2007020001,20072008,2007-09-29,1,SHOT,278,4x5,PP,ANA,...,Wrist,1,1,1,1,0,0,0.097790,17.888544,26.565051
6,7,2007020001,20072008,2007-09-29,1,SHOT,287,4x5,PP,ANA,...,Slap,1,1,1,1,0,0,0.055402,49.040799,2.337306
7,8,2007020001,20072008,2007-09-29,1,SHOT,336,4x5,PP,ANA,...,Wrist,1,1,1,1,0,0,0.057798,27.586228,43.531199
8,9,2007020001,20072008,2007-09-29,1,SHOT,367,5x5,EV,ANA,...,Slap,1,1,1,1,0,0,0.034900,39.962482,31.701430
9,10,2007020001,20072008,2007-09-29,1,SHOT,475,5x4,PP,L.A,...,Wrist,1,1,1,1,0,0,0.018001,63.976558,-24.962790


In [20]:
value[['id', 'home_team', 'away_team', 'event_team', 'strength', 'strength_2', 'shot_event', 'event_zone', 'x', 'y', 'event_distance', 'event_angle']]

,id,home_team,away_team,event_team,strength,strength_2,shot_event,event_zone,x,y,event_distance,event_angle
0,1,L.A,ANA,L.A,5x5,EV,SHOT,Off,70,11,21.954498,30.068583
1,2,L.A,ANA,ANA,5x5,EV,MISS,Off,39,-14,51.923020,-15.642246
2,3,L.A,ANA,ANA,5x5,EV,SHOT,Off,68,8,22.472205,20.854458
3,4,L.A,ANA,L.A,4x4,EV,SHOT,Off,66,22,31.827661,43.726970
4,5,L.A,ANA,ANA,4x5,PP,SHOT,Off,54,-24,42.438190,-34.438989
5,6,L.A,ANA,ANA,4x5,PP,SHOT,Off,73,8,17.888544,26.565051
6,7,L.A,ANA,ANA,4x5,PP,SHOT,Off,40,2,49.040799,2.337306
7,8,L.A,ANA,ANA,4x5,PP,SHOT,Off,69,19,27.586228,43.531199
8,9,L.A,ANA,ANA,5x5,EV,SHOT,Off,55,21,39.962482,31.701430
9,10,L.A,ANA,L.A,5x4,PP,SHOT,Off,31,-27,63.976558,-24.962790
